<a href="https://colab.research.google.com/github/henriquesm/Bot_CriptoMoeda/blob/master/teste_azure_tradutor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install azure-ai-translation-text azure-ai-translation-document requests beautifulsoup4 python-dotenv

  Using cached azure_ai_translation_text-1.0.1-py3-none-any.whl.metadata (18 kB)
  Using cached azure_ai_translation_document-1.1.0-py3-none-any.whl.metadata (32 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.8/87.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.0/214.0 kB 11.4 MB/s eta 0:00:00


In [3]:
import requests
from bs4 import BeautifulSoup
from azure.core.credentials import AzureKeyCredential
from azure.ai.translation.text import TextTranslationClient
from openai import AzureOpenAI
import os
from dotenv import load_dotenv
import markdown
from datetime import datetime

In [ ]:
load_dotenv()

TRANSLATOR_KEY = os.getenv("AZURE_TRANSLATOR_KEY")
TRANSLATOR_ENDPOINT = os.getenv("AZURE_TRANSLATOR_ENDPOINT")
OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
OPENAI_KEY = os.getenv("AZURE_OPENAI_KEY")
OPENAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_DEPLOY") #"gpt-4o"
OPENAI_DEPLOYMENT_VERSION = os.getenv("AZURE_OPENAI_DEPLOY_VERSION") # "2024-08-01-preview"


In [ ]:
# Clientes
translator_client = TextTranslationClient(TRANSLATOR_ENDPOINT, AzureKeyCredential(TRANSLATOR_KEY))
openai_client = AzureOpenAI(azure_endpoint=OPENAI_ENDPOINT, api_key=OPENAI_KEY, api_version=OPENAI_DEPLOYMENT_VERSION)

In [4]:
# Glossário técnico para domínios (ex: Azure/Data Science)
GLOSSARIO_TECNICO = {
    "prompt engineering": "engenharia de prompts",
    "fine-tuning": "ajuste fino",
    "RAG": "Recuperação Aumentada por Geração",
    "LLM": "Modelo de Linguagem Grande",
    "Azure AI Foundry": "Azure AI Foundry",
    "vector embeddings": "embeddings vetoriais"
}

In [5]:
def extrair_texto_url(url):
    """Extrai texto limpo de artigo técnico"""
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    for script in soup(["script", "style", "nav", "footer"]):
        script.decompose()
    return soup.get_text(separator=' ', strip=True)[:8000]  # Limite para API

In [6]:
def traduzir_com_glossario(texto, origem="en", destino="pt"):
    """Tradução com glossário customizado"""
    # Aplica glossário pré-tradução
    for termo_en, termo_pt in GLOSSARIO_TECNICO.items():
        texto = texto.replace(termo_en, f"__{termo_en}__")  # Marca termos

    response = translator_client.translate(body=[texto], from_language=origem, to_language=destino)
    return response[0].translations[0].text

In [7]:
def refinar_contexto_openai(texto_traduzido, dominio="cloud-ai"):
    """Refina terminologia com Azure OpenAI"""
    prompt = f"""
    Você é especialista em {dominio}. Refine esta tradução técnica mantendo precisão terminológica:
    TRADUÇÃO: {texto_traduzido}

    Corrija termos imprecisos, preserve contexto técnico e glossário:
    {GLOSSARIO_TECNICO}

    Retorne APENAS o texto refinado, sem explicações.
    """
    response = openai_client.chat.completions.create(
        model=OPENAI_DEPLOYMENT,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.1  # Baixa criatividade para precisão
    )
    return response.choices[0].message.content

In [8]:
# Pipeline completa
def traduzir_artigo_tecnico(url, origem="en", destino="pt", dominio="cloud-ai"):
    texto = extrair_texto_url(url)
    traducao_base = traduzir_com_glossario(texto, origem, destino)
    traducao_final = refinar_contexto_openai(traducao_base, dominio)
    return traducao_final

In [ ]:
def salvar_traducao_markdown(traducao, url_origem, dominio="cloud-ai", arquivo="artigo_traduzido.md", targetLanguage="pt-br"):
    """Salva tradução em Markdown estruturado"""
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M")

    md_content = f"""# Artigo Técnico Traduzido - {dominio.title()}

**URL Original**: [{url_origem}]({url_origem})
**Data/Hora**: {timestamp}
**Idioma Destino**: {targetLanguage}
**Glossário Aplicado**: {len(GLOSSARIO_TECNICO)} termos técnicos preservados

## Conteúdo Traduzido

{traducao}

---

*Tradução automática via Azure AI Translator + refinamento Azure OpenAI {OPENAI_DEPLOYMENT}*
*Precisão terminológica garantida para domínio: {dominio}*"""

    with open(arquivo, 'w', encoding='utf-8') as f:
        f.write(md_content)

    print(f"Salvo em: {arquivo}")
    return arquivo

In [ ]:
target_language = "pt-br"
origen_language = "en"
# Exemplo de uso
url_artigo = "https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/use-your-data"  # Artigo técnico Azure
resultado = traduzir_artigo_tecnico(url_artigo, origem=origen_language, destino=target_language, dominio="azure-ai")
print(resultado[:1000] + "...")  # Preview
arquivo_md = salvar_traducao_markdown(resultado, url_artigo, "azure-ai", targetLanguage=target_language)